In [0]:
import requests
import json
import pandas as pd

from pyspark.sql.functions import col, current_timestamp, lit, to_json

In [0]:
#função para obter a última rodada processada
def get_ultima_rodada_processada():
    try:
        result = spark.sql("""
            SELECT MAX(rodada) as ultima
            FROM project_data_football.pontuacao_rodada
        """).collect()[0]["ultima"]
        
        return result if result is not None else 0
    except:
        return 0


In [0]:
status_mercado = requests.get("https://api.cartola.globo.com/mercado/status").json()
rodada_atual = status_mercado["rodada_atual"]

In [0]:
# implementar função para obter a última rodada processada
ultima_processada = get_ultima_rodada_processada()

print("Última rodada processada:", ultima_processada)
print("Rodada atual da API:", rodada_atual)

rodadas_para_processar = range(ultima_processada + 1, rodada_atual + 1)

print("Rodadas que serão processadas:", list(rodadas_para_processar))

In [0]:
#loop para processar todas as rodadas 
for rodada in rodadas_para_processar:
    
    print(f"Iniciando ingestão da rodada {rodada}")
    
    # 1. Pontuação da rodada
    
    url_pontuacao = f"https://api.cartola.globo.com/atletas/pontuados/{rodada}"
    pontuacao_json = requests.get(url_pontuacao).json()
    
    atletas = pontuacao_json.get("atletas", {})

    if not atletas:
        print(f"Rodada {rodada} ainda não finalizada.")
        continue

    lista_atletas = []

    for atleta_id, dados in atletas.items():
        dados["atleta_id"] = int(atleta_id)
        lista_atletas.append(dados)

    df_pontuacao = spark.createDataFrame(pd.DataFrame(lista_atletas))

    df_pontuacao = df_pontuacao \
        .withColumn("rodada", lit(rodada)) \
        .withColumn("dt_ingestao", current_timestamp())\
        .withColumn("scout", to_json(col("scout")))


    df_pontuacao.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("project_data_football.pontuacao_rodada")
    
    # 2. Partidas da rodada
    
    url_partidas = f"https://api.cartola.globo.com/partidas/{rodada}"
    partidas_json = requests.get(url_partidas).json()

    partidas = partidas_json.get("partidas", [])

    if partidas:
        df_partidas = spark.createDataFrame(pd.DataFrame(partidas))

        df_partidas = df_partidas \
            .withColumn("placar_oficial_mandante", col("placar_oficial_mandante").cast("int")) \
            .withColumn("placar_oficial_visitante", col("placar_oficial_visitante").cast("int"))

        df_partidas = df_partidas \
            .withColumn("rodada", lit(rodada)) \
            .withColumn("dt_ingestao", current_timestamp())

        df_partidas.write \
            .format("delta") \
            .mode("append") \
            .saveAsTable("project_data_football.partidas_rodada")

    print(f"Rodada {rodada} carregada com sucesso")